# Replication Project: Descriptive Statistics

**Replication Game – Berlin, 30 October 2025 Berlin**  

---

**Institut for Replication & Freie Universität Berlin**  

**Author:** [Dominik Bursy](mailto:dominik.bursy@icloud.com)  

**Last Updated:** October 2025

---

**Reference:**  
Berazneva, Julia, and Tanya S. Byker. 2017. *Does Forest Loss Increase Human Disease? Evidence from Nigeria.* American Economic Review, 107(5), 516–521. https://doi.org/10.1257/aer.p20171132

---

## Import Packages <a class="anchor" id="packages"></a>

In [3]:
import os
from pathlib import Path

import numpy as np
import pandas as pd
import geopandas as gpd
from shapely.geometry import LineString
from datetime import timedelta
import itertools

import matplotlib.pyplot as plt
import seaborn as sns

In [4]:
## Set Root Directory
ROOT_FOLDER = str(Path().absolute().parent)
print(ROOT_FOLDER)

/Users/dominikbursy/Documents/8_PhD_New/replication_game


---

In [5]:
from shapely import wkt

# Load the CSV as a regular DataFrame
gdf_nigeria_dhs = pd.read_csv(f"{ROOT_FOLDER}/output/replication_full.csv", index_col=0)

# Convert the geometry column from WKT text to shapely geometries
gdf_nigeria_dhs["geometry"] = gdf_nigeria_dhs["geometry"].apply(wkt.loads)

gdf_nigeria_dhs = gpd.GeoDataFrame(
    gdf_nigeria_dhs,
    geometry="geometry",
    crs="EPSG:4326" 
)

In [6]:
df_final = pd.read_stata(f"{ROOT_FOLDER}/resources/113539-V1/Data_programs_readme/dta.dta")
final_columns = list(df_final.columns)

## Prepare Datasets for Comparison

### Identify LGAs per Survey Wave

In [7]:
## Exclude the 2018 wave
condition = gdf_nigeria_dhs["DHSYEAR"] != 2018 
gdf_nigeria_dhs = gdf_nigeria_dhs.loc[condition]

In [8]:
def compute_lga_overlap(df, years, lga_col='LGA', year_col='DHSYEAR'):
    """
    Compute LGA overlap categories dynamically for any list of DHS years.
    
    Parameters
    ----------
    df : pandas.DataFrame
        The dataframe containing LGA and DHSYEAR columns.
    years : list
        List of DHS years to include (e.g., [2008, 2013, 2018]).
    lga_col : str
        Name of the LGA column.
    year_col : str
        Name of the DHS year column.

    Returns
    -------
    pandas.DataFrame
        Original dataframe with two new columns:
        - 'LGA_overlap': numeric category
        - 'LGA_overlap_label': descriptive label of year combination
    """

    # Step 1: Create a boolean matrix of which LGAs have each year
    has_year = (
        df.groupby(lga_col)[year_col]
        .agg(lambda x: {year: (x == year).any() for year in years})
        .apply(pd.Series)
    )

    # Step 2: Generate all possible combinations of year overlaps
    combos = []
    for i in range(1, len(years) + 1):
        combos.extend(itertools.combinations(years, i))

    # Step 3: Assign a numeric code and label for each combination
    overlap_map = {}
    label_map = {}
    for code, combo in enumerate(combos, start=1):
        combo_set = set(combo)
        overlap_map[code] = combo_set
        label_map[code] = " & ".join(str(y) for y in combo)

    # Step 4: Function to find the correct overlap category per LGA
    def get_overlap(row):
        present_years = {y for y in years if row.get(y, False)}
        for code, combo_set in overlap_map.items():
            if present_years == combo_set:
                return code
        return np.nan

    # Step 5: Apply to each LGA
    has_year['LGA_overlap'] = has_year.apply(get_overlap, axis=1)
    has_year['LGA_overlap_label'] = has_year['LGA_overlap'].map(label_map)

    # Step 6: Merge back to the original dataframe
    df = df.merge(has_year[['LGA_overlap', 'LGA_overlap_label']], 
                  left_on=lga_col, right_index=True, how='left')

    # return df
    return df

In [9]:
gdf_nigeria_dhs["LGA_overlap"] = compute_lga_overlap(gdf_nigeria_dhs, years=[2008, 2013])["LGA_overlap"] 
df_final["LGA_overlap"] = compute_lga_overlap(df_final, years=[2008, 2013])["LGA_overlap"] 

In [10]:
condition = (df_final["LGA_overlap"] == 3)
print("Number of LGAs without missing values: ", df_final.loc[condition, "LGA"].unique().shape[0])

condition = (df_final["LGA_overlap"] == 3) | (df_final["LGA_overlap"].isna())
print("Number of LGAs with missing values: ", df_final.loc[condition, "LGA"].unique().shape[0])

display(df_final["LGA_overlap"].value_counts(dropna=False))

Number of LGAs without missing values:  408
Number of LGAs with missing values:  409


LGA_overlap
3.0    46654
1.0     7962
2.0     5216
NaN       40
Name: count, dtype: int64

In [11]:
condition = (gdf_nigeria_dhs["LGA_overlap"] == 3)
print("Number of LGAs without missing values: ", gdf_nigeria_dhs.loc[condition, "LGA"].unique().shape[0])

condition = (gdf_nigeria_dhs["LGA_overlap"] == 3) | (gdf_nigeria_dhs["LGA_overlap"].isna())
print("Number of LGAs with missing values: ", gdf_nigeria_dhs.loc[condition, "LGA"].unique().shape[0])

display(gdf_nigeria_dhs["LGA_overlap"].value_counts(dropna=False))

Number of LGAs without missing values:  408
Number of LGAs with missing values:  424


LGA_overlap
3.0    47039
1.0     7942
2.0     4865
NaN       16
Name: count, dtype: int64

In [12]:
print("Cluster that are not in the final dataframe: ",
    gdf_nigeria_dhs.loc[
    ~gdf_nigeria_dhs.DHSCLUST.isin(df_final.DHSCLUST)
, "DHSCLUST"].unique()
)
print("Cluster that are not in our dataframe: ",
    df_final.loc[
    ~df_final.DHSCLUST.isin(gdf_nigeria_dhs.DHSCLUST)
    , "DHSCLUST"].unique()
)

Cluster that are not in the final dataframe:  [nan]
Cluster that are not in our dataframe:  []


### Transform Variables into Panel Structure

In [13]:
def setup_gis_variables(df):
    """
    Sets up GIS variables: lagged forest loss, luminosity change, 
    and treecover_2000 based on DHSYEAR.

    Parameters:
    df : pandas.DataFrame
        Input dataframe containing necessary columns like 'DHSYEAR',
        'loss_<year>_pt_<target_year>MEAN', 'f16<year>_cluster_2008MEAN',
        'f18<year>_cluster_2013MEAN', 'treecover_<year>MEAN'.

    Returns:
    pandas.DataFrame
        DataFrame with new GIS variables added.
    """

    n = len(df)

    # -------------------------------
    # Initialize all columns in bulk to avoid fragmentation
    # -------------------------------
    forest_cols = [f'forest_loss_{i}' for i in range(13)]
    lumi_cols = [f'luminosity_chg_{i}' for i in range(13)]
    tree_cols = ['treecover_2000']

    init_df = pd.DataFrame(np.nan, index=df.index, columns=forest_cols + lumi_cols + tree_cols)
    df = pd.concat([df, init_df], axis=1)

    # -------------------------------
    # Convert all relevant columns to numeric
    # -------------------------------
    for col in df.columns:
        df[col] = pd.to_numeric(df[col], errors='coerce')

    # -------------------------------
    # Lagged forest loss
    # -------------------------------
    for target_year in [2008, 2013]:
        for year in range(2001, 2014):
            lag = target_year - year
            if lag < 0:
                continue
            col_name = f'loss_{year}_pt_{target_year}MEAN'
            if col_name in df.columns:
                mask = df['DHSYEAR'] == target_year
                df.loc[mask, f'forest_loss_{lag}'] = df.loc[mask, col_name]

    # -------------------------------
    # Lagged luminosity change
    # -------------------------------
    # 2008
    for yr in range(2006, 2009):
        lag = 2008 - yr
        prev = yr - 1
        col_curr = f'f16{yr}_cluster_2008MEAN'
        col_prev = f'f16{prev}_cluster_2008MEAN'
        if col_curr in df.columns and col_prev in df.columns:
            mask = df['DHSYEAR'] == 2008
            df.loc[mask, f'luminosity_chg_{lag}'] = df.loc[mask, col_curr] - df.loc[mask, col_prev]

    # 2013
    for yr in range(2011, 2014):
        lag = 2013 - yr
        prev = yr - 1
        col_curr = f'f18{yr}_cluster_2013MEAN'
        col_prev = f'f18{prev}_cluster_2013MEAN'
        if col_curr in df.columns and col_prev in df.columns:
            mask = df['DHSYEAR'] == 2013
            df.loc[mask, f'luminosity_chg_{lag}'] = df.loc[mask, col_curr] - df.loc[mask, col_prev]

    # -------------------------------
    # Treecover 2000
    # -------------------------------
    for target_year in [2008, 2013]:
        col_name = f'treecover_{target_year}MEAN'
        if col_name in df.columns:
            mask = df['DHSYEAR'] == target_year
            df.loc[mask, 'treecover_2000'] = df.loc[mask, col_name]

    # -------------------------------
    # Optional: reorder columns
    # -------------------------------
    # desired_order = forest_cols[:8] + lumi_cols[:3]
    # other_cols = [col for col in df.columns if col not in desired_order]
    # df = df[desired_order + other_cols]

    return df


In [14]:
gdf_nigeria_dhs = setup_gis_variables(gdf_nigeria_dhs)
df_final = setup_gis_variables(df_final)

---

## Desciprtive Statistics DHS

In [15]:
from scipy.stats import ttest_ind, ks_2samp

# Filter data for LGA_overlap == 3
gdf_data = gdf_nigeria_dhs[gdf_nigeria_dhs["LGA_overlap"] == 3]
df_data = df_final[df_final["LGA_overlap"] == 3]

# Variables to compare
variables = ["fever", "diarrhea", "cough", 
             "forest_loss_0", "forest_loss_1", "forest_loss_2", "forest_loss_3",
             "luminosity_chg_0", "luminosity_chg_1", "luminosity_chg_2",
             "cec_ave_pt", "ph_ave_pt", "occ_ave_pt",
             "treecover_2000"]

# gdf_data[["forest_loss_0", "forest_loss_1", "forest_loss_2", "forest_loss_3"]] *= 100
# df_data[["forest_loss_0", "forest_loss_1", "forest_loss_2", "forest_loss_3"]] *= 100

# Prepare results
results = []

for var in variables:
    # Descriptive stats
    gdf_mean = gdf_data[var].mean()
    gdf_std = gdf_data[var].std()
    gdf_count = gdf_data[var].count()
    
    df_mean = df_data[var].mean()
    df_std = df_data[var].std()
    df_count = df_data[var].count()
    
    # Statistical tests
    t_stat, t_p = ttest_ind(gdf_data[var].dropna(), df_data[var].dropna(), equal_var=False)
    ks_stat, ks_p = ks_2samp(gdf_data[var].dropna(), df_data[var].dropna())
    
    # Append results
    results.append({
        "variable": var,
        "gdf_mean": round(gdf_mean, 3),
        "gdf_std": round(gdf_std, 3),
        "gdf_count": gdf_count,
        "df_mean": round(df_mean, 3),
        "df_std": round(df_std, 3),
        "df_count": df_count,
        "t_stat": round(t_stat, 3),
        "t_p": round(t_p, 4),
        "ks_stat": round(ks_stat, 3),
        "ks_p": round(ks_p, 4)
    })

# Convert to DataFrame for easy viewing
comparison_df = pd.DataFrame(results)
comparison_df.round(3)

,variable,gdf_mean,gdf_std,gdf_count,df_mean,df_std,df_count,t_stat,t_p,ks_stat,ks_p
0,fever,0.144,0.352,41737,0.144,0.351,41409,0.135,0.892,0.000,1.0
1,diarrhea,0.106,0.308,41786,0.106,0.307,41458,0.173,0.863,0.000,1.0
2,cough,0.110,0.312,41682,0.110,0.313,41354,-0.111,0.912,0.000,1.0
3,forest_loss_0,0.001,0.002,47039,0.001,0.004,46654,-17.524,0.000,0.047,0.0
4,forest_loss_1,0.001,0.002,47039,0.001,0.003,46654,-13.245,0.000,0.052,0.0
5,forest_loss_2,0.001,0.002,47039,0.001,0.003,46654,-16.803,0.000,0.053,0.0
6,forest_loss_3,0.000,0.001,47039,0.001,0.002,46654,-17.412,0.000,0.061,0.0
7,luminosity_chg_0,-0.098,1.193,47039,0.087,1.492,46654,-20.860,0.000,0.185,0.0
8,luminosity_chg_1,0.356,1.362,47039,0.317,1.565,46654,4.049,0.000,0.113,0.0
9,luminosity_chg_2,-0.274,1.989,47039,-0.392,2.436,46654,8.143,0.000,0.085,0.0


---

## Detailes Statistical Tests

In [16]:
import scipy.stats as stats

# Extract the two samples and drop NaNs
sample1 = gdf_nigeria_dhs.loc[gdf_nigeria_dhs["LGA_overlap"] == 3, "treecover_2000"].dropna()
sample2 = df_final.loc[df_final["LGA_overlap"] == 3, "treecover_2000"].dropna()

# Check if samples have enough data and variation
if len(sample1) < 2 or len(sample2) < 2:
    print("Not enough data to perform t-test.")
elif sample1.nunique() == 1 and sample2.nunique() == 1 and sample1.iloc[0] == sample2.iloc[0]:
    print("Both samples have identical values. T-test not applicable.")
else:
    # Perform two-sided t-test (Welch's t-test)
    t_stat, p_value = stats.ttest_ind(sample1, sample2, equal_var=False)
    print("T-statistic:", t_stat)
    print("P-value:", p_value)

    # Interpretation
    if p_value < 0.05:
        print("Reject the null hypothesis: means are significantly different")
    else:
        print("Fail to reject the null hypothesis: no significant difference")


T-statistic: -38.08903656841902
P-value: 0.0
Reject the null hypothesis: means are significantly different


In [17]:
from scipy.stats import ks_2samp

# Extract the two samples and drop NaNs
sample1 = gdf_nigeria_dhs.loc[gdf_nigeria_dhs["LGA_overlap"] == 3, "treecover_2000"].dropna()
sample2 = df_final.loc[df_final["LGA_overlap"] == 3, "treecover_2000"].dropna()

# Check if samples have enough data
if len(sample1) == 0 or len(sample2) == 0:
    print("Not enough data to perform KS test.")
else:
    # Perform two-sided KS test
    ks_stat, p_value = ks_2samp(sample1, sample2, alternative='two-sided')
    print("KS statistic:", ks_stat)
    print("P-value:", p_value)

    # Interpretation
    if p_value < 0.05:
        print("Reject the null hypothesis: distributions are significantly different")
    else:
        print("Fail to reject the null hypothesis: no significant difference in distributions")


KS statistic: 0.1129307987247612
P-value: 1.1576730202218296e-260
Reject the null hypothesis: distributions are significantly different


## Descriptive Statistics Wave 2018

In [18]:
from shapely import wkt

# Load the CSV as a regular DataFrame
gdf_nigeria_dhs = pd.read_csv(f"{ROOT_FOLDER}/output/replication_full.csv", index_col=0)

# Convert the geometry column from WKT text to shapely geometries
gdf_nigeria_dhs["geometry"] = gdf_nigeria_dhs["geometry"].apply(wkt.loads)

gdf_nigeria_dhs = gpd.GeoDataFrame(
    gdf_nigeria_dhs,
    geometry="geometry",
    crs="EPSG:4326" 
)

In [19]:
gdf_nigeria_dhs["LGA_overlap"] = compute_lga_overlap(gdf_nigeria_dhs, years=[2008, 2013, 2018])["LGA_overlap"] 

condition = (gdf_nigeria_dhs["LGA_overlap"] == 3)
print("Number of LGAs without missing values: ", gdf_nigeria_dhs.loc[condition, "LGA"].unique().shape[0])

condition = (gdf_nigeria_dhs["LGA_overlap"] == 3) | (gdf_nigeria_dhs["LGA_overlap"].isna())
print("Number of LGAs with missing values: ", gdf_nigeria_dhs.loc[condition, "LGA"].unique().shape[0])

display(gdf_nigeria_dhs["LGA_overlap"].value_counts(dropna=False))

Number of LGAs without missing values:  69
Number of LGAs with missing values:  85


LGA_overlap
7.0    64259
5.0    13637
6.0     8027
4.0     2989
3.0     2769
1.0     1479
2.0      461
NaN       16
Name: count, dtype: int64

In [20]:
# Group by year and calculate mean, std, and count
condition = (gdf_nigeria_dhs["LGA_overlap"] == 7) 
gdf_nigeria_dhs[condition].groupby("DHSYEAR").agg(
    {
        "fever": ["count", "mean", "std"],
        "diarrhea": ["count", "mean", "std"],
        "cough": ["count", "mean", "std"],
    }
).round(3)


fever               diarrhea                cough              
         count   mean    std    count   mean    std  count   mean    std
DHSYEAR                                                                 
2008.0   16892  0.158  0.365    16910  0.099  0.299  16874  0.120  0.325
2013.0   22237  0.135  0.342    22260  0.109  0.312  22200  0.105  0.306
2018.0   18286  0.239  0.427    18287  0.128  0.334  18292  0.169  0.374

---